## Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import csv
from config import db_user, db_password, db_host, db_port, db_name

## Extract csv into DataFrame

In [2]:
csv_file = "data/US_Accidents_Dec20_Updated.csv"
accident_data_df = pd.read_csv(csv_file)
accident_data_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,37.165850,-121.988062,1.4,Stationary traffic on CA-17 from Summit Rd (CA...,...,False,False,False,False,False,False,Night,Night,Night,Night
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,39.110390,-119.773781,0.0,Accident on US-395 Southbound at Topsy Ln.,...,False,False,False,False,True,False,Day,Day,Day,Day
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,26.102942,-80.265091,0.0,Accident on I-595 Westbound at Exit 4 / Pine I...,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
csv_file2 = "data/WeatherEvents_Jan2016-Dec2020.csv"
weather_data_df = pd.read_csv(csv_file2)
weather_data_df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


## Transform DataFrames

### Accident DataFrame

In [13]:
transformed_accident = accident_data_df[["Start_Time", "End_Time", "Start_Lat", "Start_Lng", "Description", "Severity"]]
transformed_accident.head()
transformed_accident.count()

Start_Time     2906610
End_Time       2906610
Start_Lat      2906610
Start_Lng      2906610
Description    2906610
Severity       2906610
dtype: int64

In [14]:
#Rename columns
transformed_accident = transformed_accident.rename(columns = {"Start_Time" : "start_time",
                                                              "End_Time" : "end_time",
                                                             "Start_Lat" : "lat",
                                                             "Start_Lng" : "lng"})

transformed_accident.head()

,start_time,end_time,lat,lng,Description,Severity
0,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,Accident on Tanner Rd at Pennbrooke Ln.,2
1,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,Accident on Houston Branch Rd at Providence Br...,2
2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,Stationary traffic on CA-17 from Summit Rd (CA...,2
3,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,Accident on US-395 Southbound at Topsy Ln.,2
4,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,Accident on I-595 Westbound at Exit 4 / Pine I...,3


In [6]:
# Clean the data by dropping duplicates
transformed_accident.drop_duplicates
transformed_accident.count()

start_time     2906610
end_time       2906610
lat            2906610
lng            2906610
Description    2906610
dtype: int64

### Weather DataFrame

In [11]:
transformed_weather = weather_data_df[["StartTime(UTC)", "EndTime(UTC)", "LocationLat", "LocationLng", "Type", "Severity"]]
transformed_weather.head()
transformed_weather.count()

StartTime(UTC)    6274206
EndTime(UTC)      6274206
LocationLat       6274206
LocationLng       6274206
Type              6274206
Severity          6274206
dtype: int64

In [12]:
#Rename columns
transformed_weather = transformed_weather.rename(columns = {"StartTime(UTC)" : "start_time",
                                                            "EndTime(UTC)" : "end_time",
                                                            "LocationLat" : "lat",
                                                            "LocationLng" : "lng"})

transformed_weather.head()

,start_time,end_time,lat,lng,Type,Severity
0,2016-01-06 23:14:00,2016-01-07 00:34:00,38.0972,-106.1689,Snow,Light
1,2016-01-07 04:14:00,2016-01-07 04:54:00,38.0972,-106.1689,Snow,Light
2,2016-01-07 05:54:00,2016-01-07 15:34:00,38.0972,-106.1689,Snow,Light
3,2016-01-08 05:34:00,2016-01-08 05:54:00,38.0972,-106.1689,Snow,Light
4,2016-01-08 13:54:00,2016-01-08 15:54:00,38.0972,-106.1689,Snow,Light


In [10]:
# Clean the data by dropping duplicates
transformed_weather.drop_duplicates
transformed_weather.count()

StartTime(UTC)    6274206
EndTime(UTC)      6274206
LocationLat       6274206
LocationLng       6274206
Type              6274206
dtype: int64

## Create Database Connection

In [ ]:
# configure the connection string
rds_connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# confirm the connection string
print(rds_connection_string)
        
# connect to the database
engine = create_engine(rds_connection_string)
conn = engine.connect()